# Fraudulent User Detection Using Amazon Dataset
### Penghao Xu, Yuan Chen, Jiawei Wu, Haojing Lu

## Part 3. Proposed New Methods

This script is used to apply our proposed new model to fraudulent user detection on Amazon review dataset.


In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch import nn